In [1]:
from pathlib import Path
import open3d as o3d
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm
import pandas as pd

In [2]:
outputs=Path("/Users/amir/Library/CloudStorage/OneDrive-MassGeneralBrigham/papers/01_cardiovision/outputs/cardiovision_results")

In [3]:
cusps = list(outputs.rglob("*cusp.ply"))
calcium_file = outputs/"calcs_corr.stl"

In [4]:
right_cusp = np.asarray(o3d.io.read_triangle_mesh(str(cusps[0])))
left_cusp = np.asarray(o3d.io.read_triangle_mesh(str(cusps[1])))
non_cusp = np.asarray(o3d.io.read_triangle_mesh(str(cusps[2])))

In [5]:
calcium = np.asarray(o3d.io.read_triangle_mesh(str(calcium_file)).vertices)
landmarks = pd.read_csv(outputs/"landmarks.csv", header=None)

In [18]:
calcium_stl=np.asarray(o3d.io.read_triangle_mesh(str(calcium_file)))

## slicing
- Left cusp: p1 and p2
- right cusp: p1 and p3
- non cusp: p2 and p3

In [6]:
center = landmarks.mean().values
center[2] = landmarks.iloc[5][2]

In [7]:
landmarks = np.array(landmarks)

In [8]:
top_landmarks = landmarks[:3,:2]
bottom_landmarks = landmarks[3:-1,:2]

In [9]:
top_angels = []
for landmark in top_landmarks:
    angel = np.rad2deg(np.arctan2(landmark[1]-center[1], landmark[0]-center[0]))
    top_angels.append(angel)
bottom_angels=[]
for landmark in bottom_landmarks:
    angel = np.rad2deg(np.arctan2(landmark[1]-center[1], landmark[0]-center[0]))
    if angel<0:
        angel+=360
    bottom_angels.append(angel)

In [10]:
calcium_right = []
calcium_left = []
calcium_none=[]

counter = 0
for point in calcium:
    angel = np.rad2deg(np.arctan2(point[1]-center[1], point[0]-center[0]))
    if angel<0:
        angel+=360
    counter+=1
    
    targets=[]
    
    # Left cusp
    if bottom_angels[0] < bottom_angels[1]:
        if bottom_angels[0] <= angel <= bottom_angels[1]:
            targets.append("left")
    else:
        if bottom_angels[1] <= angel <= bottom_angels[0]:
            targets.append("left")
            
    # None cusp      
    if bottom_angels[1] < bottom_angels[2]:
        if bottom_angels[1] <= angel <= bottom_angels[2]:
            targets.append("none")
    else:
        if bottom_angels[2] <= angel <= bottom_angels[1]:
            targets.append("none")
                
    # Right cusp
    if bottom_angels[0] < bottom_angels[2]:
        if bottom_angels[0] <= angel <= bottom_angels[2]:
            targets.append("right")
    else:
        if angel <= bottom_angels[2]:
            targets.append("right")
            
    if "left" in targets:
        calcium_left.append(point)
    elif "right" in targets:
        calcium_right.append(point)
    elif "none" in targets:
        calcium_none.append(point)

In [11]:
calcium_right_pcd = o3d.geometry.PointCloud()
calcium_right_pcd.points = o3d.utility.Vector3dVector(calcium_right)
o3d.io.write_point_cloud("./calcium_right.ply", calcium_right_pcd)

calcium_left_pcd = o3d.geometry.PointCloud()
calcium_left_pcd.points = o3d.utility.Vector3dVector(calcium_left)
o3d.io.write_point_cloud("./calcium_left.ply", calcium_left_pcd)

calcium_none_pcd = o3d.geometry.PointCloud()
calcium_none_pcd.points = o3d.utility.Vector3dVector(calcium_none)
o3d.io.write_point_cloud("./calcium_none.ply", calcium_none_pcd)

True

In [23]:
calcium_stl.clip()

ValueError: One of max or min must be given